## Imports

In [ ]:
import os
import pandas as pd
import os
import shutil
import numpy as np

## Find relevant sub-directories within data directory that have tiffs

In [2]:
data_directory = r'Z:\Rei\Post-hoc imaging' #the parent data directory
directories = []
for root, dirs, files in os.walk(data_directory):
    if ('RGECO-GCaMP' in root) & ('tiff_files' in root): #the substrings that the sub-directories of interest should contain
        print(root)
        directories.append(root)

Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\four\A4_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\four\A5_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\muji\A7_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\muji\A7_PPC_20X.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\muji\A8_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\muji\A8_PPC_20X.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\one\A6_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\one\A7_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\two\A5_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\two\A6_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\two\A6_PPC_20X.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch1\two\A7_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch2\five\A4_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RGECO-GCaMP_Batch2\five\A5_POm_PPC.tiff_files
Z:\Rei\Post-hoc imaging\RG

## Extract x- and y-positions of tiles from tiff filenames and move files to new directory

In [4]:
## Loop through all the sub-directories identified in the previous cell and determine which files need to be stitched ##
for i, (d) in enumerate(directories):
    if i == 0:
        None
    else:
        root = ('\\').join(d.split('\\')[:-1]) #parent directory of the current sub-directory (i.e. one level up)
        file = d.split('\\')[-1].split('_files')[0] #what the stitched tiff filename should be
        if file in os.listdir(root): 
            None
            
        ## If the stitched tiff doesn't exist in the parent directory ##
        else:
            for color in ['1','0']: #loop through each color channel
                new_dir = file+'_color'+color #define new folder name where files will be copied to
                if new_dir in os.listdir(root):
                    None
                else:
                    os.mkdir(root+'\\'+new_dir) #create new folder if it doesn't exist in the parent directory
                    
                dfs = []                
                ## Loop through all the individual tile tiffs in the current sub-directory ##
                for f in os.listdir(d):
                    if ('.xml' not in f) & ('.tiff' in f) & ('color' not in f): #files must satisfy these criteria
                        c = f.split('b0v0t0z0c')[1][0] #color channel identifier from tiff filename
                        if c == color:
                            x = int(f.split('b0v0t0z0c'+c+'x')[1].split('-')[0]) #x-position identifier from tiff filename
                            y = int(f.split('-512y')[1].split('-')[0]) #y-position identifier from tiff filename
                            
                            ## Create dataframe of x- and y-position with "{tiff_filename} ; ;" as index ## 
                            tile_df = pd.DataFrame([[x,y]],columns=['x','y'],index=[f+'; ;'])
                            dfs.append(tile_df) #append each tile tiff to list
                            shutil.copyfile(d+'\\'+f, root+'\\'+new_dir+'\\'+f) #copy each tile tiff to new directory
                df = pd.concat(dfs) #concatenate list of individual tiff dataframes into a single dataframe
                
                ## Create final dataframe in the correct format, save to .txt file, and copy to new directory ##
                new_dfs = ['dim = 2', ''] #first 2 lines of .txt file where "dim" refers to the number of position locators
                for index, rows in df.iterrows():
                    new_dfs.append(index+' ('+str(rows.x)+','+str(rows.y)+')')
                new_df = pd.DataFrame(new_dfs, columns=['text'])
                np.savetxt(root+'\\'+new_dir+'\\tiles.txt', new_df.values, fmt='%s')